<a href="https://colab.research.google.com/github/Nukaraju2003/DR.Abhinav-Dayal-Sir/blob/main/automates_the_task_of_evaluating_answers_to_questionsMam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 758.3 kB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import csv
# import openai
df = pd.read_csv("/content/drive/MyDrive/Jyothi Mam/first/speakers_continuous_text.csv")
print(df.head())

                                     SPEAKER_01_text  \
0                            Tell me about yourself.   
1                     What is your role in projects?   
2  Like what technologies you have used for devel...   
3  Technology in the sense you should mention whi...   
4  Okay, is growing of technology is good for hum...   

                                     SPEAKER_00_text  
0  Good afternoon, ma'am. I'm Thadala Veerabargir...  
1         My role is developing the application map.  
2  I have used libraries like NumPy, Pandas and O...  
3                                I have used Python.  
4  Yes ma'am, growing of technology is good for h...  


# few shot prompt

In [ ]:
# Few-shot prompting examples for scoring
example_scores_1 = """
For the question, "Tell me about yourself.", give a Score out of 100 each for 'Relevance', 'Clarity', 'Correctness', 'StructuredAnswers', 'Fluency', 'Professionalism', 'NoFillers', 'Focused', 'Authentic', 'Overall' for the following answer: "Hello sir,/mam. Thank you for you giving me a chance for introducing myself. I am Vishal Rathod. I am from Rathod Nagar (Shivani) in Jalan district. I have done my completed graduation in BSC at Aurangabad University. Then I completed my computer typing in English 30 40 and Marathi 30. I belong to a middle-class family, my family we are 5 members including me. My father is a farmer and my mother is a housewife. My short-term goal is to get a job in a company I can improve my computer experience and skill. My long-term goal and dream I work in the banking sector so I can build my career and help to the organization. My strength is I am a self-motivated, hardworking and honest person. My hobby is driving, swimming and playing cricket. That's all about me. And.Thank you for giving me a chance I am so glad."

Scores:
Relevance: 60
Clarity: 70
Correctness: 65
StructuredAnswers: 55
Fluency: 60
Professionalism: 55
NoFillers: 65
Focused: 50
Authentic: 75
Overall: 60
"""

example_scores_2 = """
For the question, "What is data structure?", give a Score out of 100 each for 'Relevance', 'Clarity', 'Correctness', 'StructuredAnswers', 'Fluency', 'Professionalism', 'NoFillers', 'Focused', 'Authentic', 'Overall' for the following answer: "A data structure is a way of organizing data that considers not only the items stored, but also their relationship to each other. Advance knowledge about the relationship between data items allows designing of efficient algorithms for the manipulation of data."

Scores:
Relevance: 95
Clarity: 90
Correctness: 100
StructuredAnswers: 90
Fluency: 90
Professionalism: 95
NoFillers: 95
Focused: 95
Authentic: 90
Overall: 93
"""


# Example 1 of overall feedback
example_feedback_1 = """
Here are the answers to several questions:

Q1: "Why should we hire you?"
A1: "I bring a unique combination of skills and experience that align with your company's needs. My background in data science and strong problem-solving abilities make me a great fit for this role."

Q2: "What is your biggest weakness?"
A2: "I tend to be a perfectionist, which sometimes leads me to spend more time on tasks than necessary. However, I am working on balancing quality with efficiency."

Overall Feedback:
The candidate provides clear and relevant answers, with strong alignment to the role and company values. However, the answer to the second question could have been structured better, and the candidate should avoid dwelling too much on weaknesses. The response flow is smooth, and the candidate’s professionalism is evident throughout. Key strengths were well-highlighted, but there is room for improvement in balancing confidence with humility.
"""

# Example 2 of overall feedback
example_feedback_2 = """
Here are the answers to several questions:

Q1: "Tell me about a time you faced a challenge at work."
A1: "In my previous role, I was tasked with leading a project under a tight deadline. I organized the team, delegated tasks effectively, and ensured open communication, which resulted in the successful completion of the project ahead of schedule."

Q2: "How do you handle stress?"
A2: "I manage stress by maintaining a balanced work-life routine and practicing mindfulness. I also prioritize tasks effectively to avoid last-minute rushes."

Overall Feedback:
The candidate demonstrates strong leadership and problem-solving skills, particularly in high-pressure situations. Their answers are structured and concise, showing clarity and focus. However, they could enhance their responses by providing more specific examples or metrics to quantify their achievements. The candidate’s approach to stress management is practical, but could benefit from more detail on how it impacts their work performance.
"""




#Scoring

In [ ]:
def openapicall(prompt):
  model_engine = "gpt-3.5-turbo"
  response = openai.ChatCompletion.create(
      model= model_engine,
      temperature=0.5,
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt},
      ])
  return response


In [ ]:
# Function to extract numerical scores from the response
def extract_scores(text):
    scores = re.findall(r"(\w+): (\d{1,3})", text)
    result = {category: 0 for category in ['Relevance', 'Clarity', 'Correctness', 'StructuredAnswers', 'Fluency', 'Professionalism', 'NoFillers', 'Focused', 'Authentic', 'Overall']}

    for category, value in scores:
        standardized_category = category.capitalize() if category.lower() != 'structuredanswers' else 'StructuredAnswers'
        standardized_category = standardized_category if standardized_category in result else None

        if standardized_category:
            result[standardized_category] = int(value)

    return result


In [ ]:
import csv
import openai
import re

openai.api_key = ""   #openai api key

# Open the CSV file to read data
with open('/content/drive/MyDrive/Jyothi Mam/first/speakers_continuous_text.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)
    with open('/content/drive/MyDrive/Jyothi Mam/first/scores.csv', mode='w', newline='') as scores_file:
        fieldnames = ['Relevance', 'Clarity', 'Correctness', 'StructuredAnswers', 'Fluency', 'Professionalism', 'NoFillers', 'Focused', 'Authentic', 'Overall']
        writer = csv.DictWriter(scores_file, fieldnames=fieldnames)
        writer.writeheader()

        for i, row in enumerate(csv_reader, 1):
            column1_value = row[0]
            column2_value = row[1]
            prompt = f"""\
             {example_scores_1}\n\n
             {example_scores_2}\n\n
             For the question, "{column2_value}", give a Score out of 100 each for 'Relevance', 'Clarity', 'Correctness', 'StructuredAnswers', 'Fluency', 'Professionalism', 'NoFillers', 'Focused', 'Authentic', 'Overall' for the following answer: "{column1_value}".
             Use the provided examples as a reference to ensure that each parameter is thoroughly evaluated. The scoring should reflect the quality of the response based on how well it addresses the question and meets the specified criteria. Ensure that no parameter is overlooked and that the evaluation is comprehensive and accurate."""
            response = openapicall(prompt)
            message = response.choices[0]['message']['content']

            print(message)
            scores = extract_scores(message)
            print(f"Scores for row {i}: {scores}")
            writer.writerow(scores)
            print(f"Scores written for row {i}")

    print("All scores extracted and saved to scores.csv")


Scores for the third response:

Relevance: 95
Clarity: 90
Correctness: 100
StructuredAnswers: 90
Fluency: 95
Professionalism: 95
NoFillers: 95
Focused: 95
Authentic: 90
Overall: 94

This response provides relevant and detailed information about the individual's educational background, family, interests, skills, and projects. The answer is clear, correct, and well-structured, with a good flow of information. The language used is professional, free of unnecessary fillers, and focused on key points. The response comes across as authentic and genuine, showcasing the individual's achievements and aspirations effectively. Overall, it is a strong and well-rounded introduction.
Scores for row 1: {'Relevance': 95, 'Clarity': 90, 'Correctness': 100, 'StructuredAnswers': 90, 'Fluency': 95, 'Professionalism': 95, 'NoFillers': 0, 'Focused': 95, 'Authentic': 90, 'Overall': 94}
Scores written for row 1
Scores:
Relevance: 80
Clarity: 85
Correctness: 100
StructuredAnswers: 80
Fluency: 90
Professionalis

# overall feedback

In [ ]:
def openapicallfeedback(prompt):
  model_engine = "gpt-3.5-turbo"
  overall_feedback_response = openai.ChatCompletion.create(
      model= model_engine,
      temperature=0.5,
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": overall_feedback_prompt},
      ])
  return overall_feedback_response

In [ ]:

all_answers = []
with open('/content/drive/MyDrive/Jyothi Mam/first/speakers_continuous_text.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)
    for row in csv_reader:
        column1_value = row[0]
        column2_value = row[1]
        all_answers.append(f"Q: \"{column2_value}\" A: \"{column1_value}\"")

combined_answers = "\n\n".join(all_answers)

# few shot prompt for overall feedback
overall_feedback_prompt = f"""\
{example_feedback_1}\n\n
{example_feedback_2}\n\n
Based on the feedback provided for the question and answers, please provide an overall feedback summary. The summary should include two distinct paragraphs:

1. **Strengths**: Praise the candidate's responses, highlighting any positive aspects such as clear communication, relevant content, or effective presentation of their skills and knowledge.

2. **Areas for Improvement**: Address the areas that need improvement. Focus on aspects such as clarity, relevance, structure, and professionalism, and provide actionable suggestions for how the candidate can enhance their performance in future interviews.

**{combined_answers}"""

overall_feedback_response = openapicallfeedback(overall_feedback_prompt)
overall_feedback = overall_feedback_response.choices[0]['message']['content'].strip()

print("Overall Feedback:" , overall_feedback)

# Write overall feedback to a text file
with open('/content/drive/MyDrive/Jyothi Mam/first/overall_feedback.txt', mode='w') as feedback_file:
    feedback_file.write(overall_feedback)

print("Overall feedback written to overall_feedback.txt")


Overall Feedback: **Strengths**: The candidate demonstrates a strong educational background and a clear interest in technology, particularly in programming languages like Java and Python. They have completed notable projects showcasing their practical skills in software development. Additionally, the candidate shows willingness and adaptability to work in different shifts, reflecting a flexible approach to job requirements. Their responses, though sometimes brief, indicate a foundational understanding of technical concepts, and their interest in learning and growing in the IT sector is evident.

**Areas for Improvement**: The candidate should work on providing more detailed and specific responses to technical questions, especially when asked about programming concepts and languages. It would be beneficial for them to enhance their knowledge and understanding of technical terms and principles to communicate effectively in interviews. Additionally, the candidate could benefit from struct